# Workflow for SOC modelling

In [2]:
import numpy as np
import pandas as pd

## Colin's idea

This section describes what (I think) Colin had intended to do, based on his soil carbon primer and the datasets that he provided.  The soil team has already made some progress on this.

### Soil carbon data
Soil carbon values from field samples have been processed and put into tabular form, like:

In [13]:
lons = np.array([23, 36.12, 176])
lats = np.array([34., -34, 78.])
time = np.array([1976, 1987, 2010])
socs = np.array([0.002, 0.13, 0.45])

df = pd.DataFrame({'lon':lons, 'lat':lats, 'time':time, 'SOC':socs})
df

,lon,lat,time,SOC
0,23.00,34.0,1976,0.002
1,36.12,-34.0,1987,0.130
2,176.00,78.0,2010,0.450


### Environmental covariates

We also have a stack of about 150 environmental covariates, based on a list provided by Coin, uploaded onto the Google Earth Engine.  Each covariate is in the form of a raster over the globe. e.g. EarthEnv's Eastness:

<img src="earthenv_eastness.png" width=300>

As they are, each covariate is independent of time and only varies with location (longitude, latitude).

### Modelling

The soil carbon table and the covariates can be merged by location, `(lon, lat)`, yielding tabular data like the following:

In [14]:
avg_temp_jan = np.array([2.01, 0.02, .2])
SG_SOC_Stock = np.array([0.001, 0.3, 0.4])
WCS_Human_Footprint_2009 = np.array([0.5, 0.7, 0.7])
earthenv_landcover_shrubs = np.array([0.1, 0.3, 0.3])

df['avg_temp_jan'] = avg_temp_jan
df['SG_SOC_Stock'] = SG_SOC_Stock
df['WCS_Human_Footprint_2009'] = WCS_Human_Footprint_2009
df['earthenv_landcover_shrubs'] = earthenv_landcover_shrubs

df = df[['avg_temp_jan', 'SG_SOC_Stock', 'WCS_Human_Footprint_2009', 
         'earthenv_landcover_shrubs', 'lon', 'lat', 'time', 'SOC']]

df

,avg_temp_jan,SG_SOC_Stock,WCS_Human_Footprint_2009,earthenv_landcover_shrubs,lon,lat,time,SOC
0,2.01,0.001,0.5,0.1,23.00,34.0,1976,0.002
1,0.02,0.300,0.7,0.3,36.12,-34.0,1987,0.130
2,0.20,0.400,0.7,0.3,176.00,78.0,2010,0.450


For illustration purposes, only 4 covariates are shown here.  They are:
- `avg_temp_jan` --- Average January temperature.
- `SG_SOC_Stock` --- SoilGrid soil organic carbon stock 2017.
- `WCS_Human_Footprint_2009` --- Wildlife Conservation Society's 2009 human footprint
- `earthenv_landcover_shrubs` -- EarthEnv's shrubs landcover.

In such a table, taking SOC as the dependent variable and everything else as the independent variables, we can train/fit a model of the form:

$$
y = f(\phi, \theta, t) = g(x(\phi, \theta)) \: h(t)
$$

because the covariates are time independent.  $y$ is the SOC and $x$ are the covariates.  

For data of this tabular form, possible algorithms include:
- Quantile random forest
- Tabnet/NNs
- Interpretable gradient boosting machines

## Improving the covariate stack

The following are ways the covariate stack can be improved:

1. **Use correct, past-only histories.**  
   For example, take the covariate `avg_temp_jan`.  It's the average January temperature that is averaged over the years 1970-2000.  For a SOC sample taken in 1976, it doesn't make physical sense to try to predict it using temperatures from 1976-2000.  For a sample taken in 2010, this covariate doesn't take into account the history between 2000-2010.  In this case, it's possible to dig out the correct temperature history from historical climate data.  In general, climate variables, such as temperature, cloud, and precipitation should be available out there at a decent enough temporal resolution.

2. **Supplment with time-varying remote sensing data/satellite images.**  
   For example, it's expected that human development varies with time, but the covariate `WCS_Human_Footprint_2009` is human footprint as of the year 2009, so issues related to incompatible history also apply here, and it's also the same with `earthenv_landcover_shrubs`, EarthEnv's shrubs landcover, which its description says is derived other data products published in 2015.  Information about these variables are encoded in satellite images, which are available at many more timestamps, so for each SOC sample, we can also grab the corresponding history and use that as a covariate.  e.g. LANDSAT has been observing from 1970 to the present.  

After these changes, the tabular data might now look like:

In [20]:
df_plus = df.copy()
df_plus['LANDSAT Band 1'] = np.array([.33, .56, .2])
df_plus['LANDSAT Band 2'] = np.array([45., 12., 187.])
df_plus.loc[:, 'avg_temp_jan'] = np.array([1.1, 0.3, 4])

df_plus[list(df_plus.columns[-2:]) + list(df_plus.columns[:-2])]

,LANDSAT Band 1,LANDSAT Band 2,avg_temp_jan,SG_SOC_Stock,WCS_Human_Footprint_2009,earthenv_landcover_shrubs,lon,lat,time,SOC
0,0.33,45.0,1.1,0.001,0.5,0.1,23.00,34.0,1976,0.002
1,0.56,12.0,0.3,0.300,0.7,0.3,36.12,-34.0,1987,0.130
2,0.20,187.0,4.0,0.400,0.7,0.3,176.00,78.0,2010,0.450


Notice that `avg_temp_jan` values are different from before because we using January temperatures that are averaged over different periods.  For sample 0, over past-1976. For sample 1, past-1987, and so on.  

The `LANDSAT Band 1` value for sample 0 would be the value of the LANDSAT pixel at sample 0's location in spectral band 1, averaged over the years past-1987.  

Using such a dataset, the resulting model can interpreted as of the form:

$$
y = g(x) \: h(t)
$$

Some covariates will remain static $x(\phi, \theta)$, but some will have time-dependence, and of the form:

$$
x(\phi, \theta, t) = \frac{1}{T} \sum_{t' = t - T}^{t} x(\phi, \theta, t')
$$

because the average history is used.  $T$ is the length of history.

Given the data is still tabular, suitable algorithms remain the same. 

## Modelling using image-form data

Instead of using only information at each field sample's point location, the area surrounding it can also be used to train the models.  

Such a dataset can be generated in GEE, by giving the `ee.filterBounds` method an area geometry instead of a point geometry.  Each value in the tabular data above will now become a 2-d array, or an image.  

Suitable algorithms for this type of data:
- Convolutional neural networks.

**Couldn't we just use existing soil carbon maps out there?**

1. In the covariate stack, there are variables like SoilGrid SOC 0-30 cm, SoilGrid_SOC 30-60 cm, etc.  These are soil carbon values. for each point on Earth.  That these are in the covariate stack suggests that SoilGrid have more to be desired of, otherwise we could just take these values and that would be the end of story.  
2. This is also supported by this Slack post by Colin.